Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt
import os

In [2]:
#pickle_file = 'notMNIST.pickle'
pickle_file = 'notMNIST_noDupNorOvlp.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (438087, 28, 28) (438087,)
Validation set (15513, 28, 28) (15513,)
Test set (13649, 28, 28) (13649,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (438087, 28, 28, 1) (438087, 10)
Validation set (15513, 28, 28, 1) (15513, 10)
Test set (13649, 28, 28, 1) (13649, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 1001

#Plots
p_freq = 10
losses = []
accuracies = []
rates = []

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #Plot variables
    if (step % p_freq == 0):
       losses.append(l)
       accuracies.append(accuracy(valid_prediction.eval(), valid_labels))
       rates.append(0.05)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

# Show the results.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plt.subplots_adjust(wspace=.8)
fig.set_size_inches(10, 4)
ax1.plot(range(0, num_steps, p_freq), losses)
ax1.set_ylabel("Loss")
ax1.set_xlabel("Training steps")
ax2.plot(range(0, num_steps, p_freq), accuracies)
ax2.set_ylabel("Validation accuracy")
ax2.set_xlabel("Training steps")
ax3.plot(range(0, num_steps, p_freq), rates)
ax3.set_ylabel("Learning rate")
ax3.set_xlabel("Training steps")
ax3.set_ylim([0, 1])
plt.show()

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [ ]:
batch_size = 256
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Convolution 1 Layer
  # Input channels: num_channels = 1
  # Output channels: depth = 16
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  # Convolution 2 Layer
  # Input channels: depth = 16
  # Output channels: depth = 16
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
  # Fully Connected Layer (Densely Connected Layer)
  # Use neurons to allow processing of entire image
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
  # Readout layer: Softmax Layer
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    # ksize is kernel size = [1, 2, 2, 1]
    padd1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(padd1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    padd2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = padd2.get_shape().as_list()
    reshape = tf.reshape(padd2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 1001

#Plots
p_freq = 10
losses = []
accuracies = []
rates = []

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #Plot variables
    if (step % p_freq == 0):
       losses.append(l)
       accuracies.append(accuracy(valid_prediction.eval(), valid_labels))
       rates.append(0.05)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

# Show the results.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plt.subplots_adjust(wspace=.8)
fig.set_size_inches(10, 4)
ax1.plot(range(0, num_steps, p_freq), losses)
ax1.set_ylabel("Loss")
ax1.set_xlabel("Training steps")
ax2.plot(range(0, num_steps, p_freq), accuracies)
ax2.set_ylabel("Validation accuracy")
ax2.set_xlabel("Training steps")
ax3.plot(range(0, num_steps, p_freq), rates)
ax3.set_ylabel("Learning rate")
ax3.set_xlabel("Training steps")
ax3.set_ylim([0, 1])
plt.show()

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [ ]:
batch_size = 512
patch_size1 = 5
patch_size2 = 3
depth1 = 6
depth2 = 12
num_hidden_3 = 256
beta = 0.0
num_steps = 501

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Convolution 1 Layer
  # Input channels: num_channels = 1
  # Output channels: depth = 16
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size1, patch_size1, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))

  # Convolution 2 Layer
  # Input channels: depth = 16
  # Output channels: depth = 16
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size2, patch_size2, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
  # Fully Connected Layer (Densely Connected Layer)
  # Use neurons to allow processing of entire image
  layer3_weights = tf.Variable(tf.truncated_normal(
      [6*6*12, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
  # Readout layer: Softmax Layer
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden1 = tf.nn.dropout(tf.nn.relu(conv1 + layer1_biases), 0.5)
    # ksize is kernel size = [1, 2, 2, 1]
    padd1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(padd1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.dropout(tf.nn.relu(conv2 + layer2_biases), 0.5)
    padd2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = padd2.get_shape().as_list()
    reshape = tf.reshape(padd2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),0.5)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  regularization = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) \
    + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights)
  loss = tf.reduce_mean(loss + beta * regularization)
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 2500, 0.1)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
#Plots
p_freq = 2
losses = []
accuracies = []
rates = []

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #Plot variables
    if (step % p_freq == 0):
       losses.append(l)
       accuracies.append(accuracy(valid_prediction.eval(), valid_labels))
       rates.append(learning_rate.eval())
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

# Show the results.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plt.subplots_adjust(wspace=.8)
fig.set_size_inches(10, 4)
ax1.plot(range(0, num_steps, p_freq), losses)
ax1.set_ylabel("Loss")
ax1.set_xlabel("Training steps")
ax2.plot(range(0, num_steps, p_freq), accuracies)
ax2.set_ylabel("Validation accuracy")
ax2.set_xlabel("Training steps")
ax3.plot(range(0, num_steps, p_freq), rates)
ax3.set_ylabel("Learning rate")
ax3.set_xlabel("Training steps")
ax3.set_ylim([0, 0.1])
plt.show()

Second approach

In [20]:
batch_size = 1025
patch_size1 = 9
patch_size2 = 7
depth1 = 25
depth2 = 80
num_hidden1 = 1728
num_hidden2 = 864
num_hidden3 = 200
beta = 0.0001
num_steps = 1001

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Convolution 1 Layer
  # Input channels: num_channels = 1
  # Output channels: depth = 16
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size1, patch_size1, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))

  # Convolution 2 Layer
  # Input channels: depth = 16
  # Output channels: depth = 16
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size2, patch_size2, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
  # Fully Connected Layer (Densely Connected Layer)
  # Use neurons to allow processing of entire image
  layer3_weights = tf.Variable(tf.truncated_normal(
      [7*7*depth2, num_hidden1], stddev=0.01))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))
    
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden1, num_hidden2], stddev=0.01))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
    
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_hidden3], stddev=0.01))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden3]))
    
  # Readout layer: Softmax Layer
  layer6_weights = tf.Variable(tf.truncated_normal(
      [num_hidden3, num_labels], stddev=0.01))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.dropout(tf.nn.relu(conv1 + layer1_biases), 0.75)
    # ksize is kernel size = [1, 2, 2, 1]
    padd1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(padd1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.dropout(tf.nn.relu(conv2 + layer2_biases), 0.75)
    padd2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = padd2.get_shape().as_list()
    reshape = tf.reshape(padd2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),0.75)
    hidden2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1, layer4_weights) + layer4_biases),0.75)
    hidden3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden2, layer5_weights) + layer5_biases),0.9)
    return tf.matmul(hidden3, layer6_weights) + layer6_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  regularization = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) \
    + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer5_weights)\
    + tf.nn.l2_loss(layer6_weights)
  loss = tf.reduce_mean(loss + beta * regularization)
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 2000, 0.1)
  optimizer = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Initialized
Minibatch loss at step 0: 2.384114
Minibatch accuracy: 9.1%
Validation accuracy: 10.5%
Minibatch loss at step 50: 0.656428
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 100: 0.619350
Minibatch accuracy: 82.1%
Validation accuracy: 85.1%
Minibatch loss at step 150: 0.502779
Minibatch accuracy: 86.1%
Validation accuracy: 87.3%
Minibatch loss at step 200: 0.430326
Minibatch accuracy: 89.0%
Validation accuracy: 88.0%
Minibatch loss at step 250: 0.438589
Minibatch accuracy: 87.9%
Validation accuracy: 88.6%
Minibatch loss at step 300: 0.383042
Minibatch accuracy: 90.3%
Validation accuracy: 88.9%
Minibatch loss at step 350: 0.387351
Minibatch accuracy: 90.0%
Validation accuracy: 88.8%
Minibatch loss at step 400: 0.334925
Minibatch accuracy: 91.4%
Validation accuracy: 89.5%
Minibatch loss at step 450: 0.438181
Minibatch accuracy: 89.6%
Validation accuracy: 89.7%
Minibatch loss at step 500: 0.412943
Minibatch accuracy: 89.5%
Validation accuracy: 90.1%
Mi

In [21]:
#Plots
p_freq = 10
losses = []
accuracies = []
rates = []

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #Plot variables
    if (step % p_freq == 0):
       losses.append(l)
       accuracies.append(accuracy(valid_prediction.eval(), valid_labels))
       rates.append(learning_rate.eval())
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

# Show the results.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plt.subplots_adjust(wspace=.8)
fig.set_size_inches(10, 4)
ax1.plot(range(0, num_steps, p_freq), losses)
ax1.set_ylabel("Loss")
ax1.set_xlabel("Training steps")
ax2.plot(range(0, num_steps, p_freq), accuracies)
ax2.set_ylabel("Validation accuracy")
ax2.set_xlabel("Training steps")
ax3.plot(range(0, num_steps, p_freq), rates)
ax3.set_ylabel("Learning rate")
ax3.set_xlabel("Training steps")
ax3.set_ylim([0, 0.2])
plt.show()